Only need data/passage_embedding_dl_ir.h5

In [ ]:
!git clone https://github.com/team21-2023/Team_21_InterIIT.git
!mv Team_21_InterIIT data
!mv data/* .
!rm -rf data

Cloning into 'Team_21_InterIIT'...
remote: Enumerating objects: 9, done.
remote: Counting objects: 100% (9/9), done.
remote: Compressing objects: 100% (8/8), done.
remote: Total 9 (delta 0), reused 9 (delta 0), pack-reused 0
Unpacking objects: 100% (9/9), 3.60 MiB | 9.86 MiB/s, done.


Install Packages for Inference

In [ ]:
!pip install -U -q sentence-transformers
!pip install -q gensim==4.1.2
!pip install elasticsearch==7.9.1
!pip install -q sentencepiece
!pip install -q transformers 
!pip install -q datasets optimum onnx onnxruntime onnxruntime_tools

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


Importing libraries

In [ ]:
# Allowed to make changes.
import collections
import json
import string
import timeit

# Allowed to make changes.
import collections
import string
import timeit

#imports for data processing
import pandas as pd
import numpy as np
import json
from ast import literal_eval
import re
from tqdm import tqdm
from google.colab import files
from numpy import genfromtxt
from functools import reduce

#imports for modelling
import os
import sys
import time
import torch
import h5py
import pickle
from tqdm import tqdm
from sentence_transformers import SentenceTransformer, util
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.neighbors import NearestNeighbors
import warnings
warnings.filterwarnings("ignore")
warnings.filterwarnings(action='ignore', category=UserWarning, module='gensim')
import gensim
from gensim.models import LsiModel, TfidfModel
from gensim.similarities import MatrixSimilarity
from gensim.corpora import Dictionary
from gensim.parsing.preprocessing import preprocess_documents, preprocess_string
from elasticsearch import Elasticsearch
from transformers import AutoModelForQuestionAnswering, pipeline, AutoTokenizer
from transformers import QuestionAnsweringPipeline
from optimum.onnxruntime import ORTModelForQuestionAnswering
from optimum.onnxruntime import ORTQuantizer
from optimum.onnxruntime.configuration import AutoQuantizationConfig
from optimum.onnxruntime import ORTModelForFeatureExtraction
from transformers import Pipeline
import torch.nn.functional as F

In [ ]:
paragraphs_data = pd.read_csv("paragraphs.csv", encoding="utf-8")
themes = paragraphs_data.theme.unique()
theme_weights = {i:1/len(themes) for i in themes}
#Adding theme index
theme_to_id_dict = {}
for i in range(len(themes)):
    t = themes[i]
    theme_to_id_dict[t]=i
paragraphs_data['tid'] = paragraphs_data.apply(lambda row: theme_to_id_dict[row.theme], axis=1)
paragraphs_data['id'] -= 1 #just for paragraphs.csv

In [ ]:
# Allowed to make changes.
def get_theme_model(theme):
  # Here, you can load and return a model which is fine tuned for a theme.
  # In case, you only have a global model, you can return that.
  #return global_model
  return 0

IR related functions

In [ ]:
#TF-IDF Config
# defining the TF-IDF
tfidf_configs = {
    'lowercase': True,
    'analyzer': 'word',
    'stop_words': 'english',
    'binary': True,
    'max_df': 0.9,
    'max_features': 10_000
}
# defining our pipeline
embedding = TfidfVectorizer(**tfidf_configs)

In [ ]:
def tfidf(questions, theme, tid, k):
  top_k_tfidf = []
  # defining the number of documents to retrieve
  retriever_configs = {
      'n_neighbors': k,
      'metric': 'cosine'
  }
  # defining our pipeline
  retriever = NearestNeighbors(**retriever_configs)
  # let's train theme based model to retrieve the document id
  t_paragraphs_data = paragraphs_data[paragraphs_data.tid==tid]
  t_para = t_paragraphs_data.paragraph.to_list()
  t_pid = t_paragraphs_data.id.to_list()
  X = embedding.fit_transform(t_paragraphs_data['paragraph'])
  retriever.fit(X, t_paragraphs_data['id'])
  # evaluate
  t_ques = [questions[i]['question'] for i in range(len(questions))]
  X = embedding.transform(t_ques)
  t_y_pred = retriever.kneighbors(X, return_distance=True)
  #re-map indices to pid
  t_y_pred_pid = np.array(list(map(lambda x: [t_pid[i] for i in x], t_y_pred[1])))
  top_k_tfidf = [1-t_y_pred[0], t_y_pred_pid]
  return top_k_tfidf

In [ ]:
def lsi(questions, theme, tid, k):
  top_k_lsi = []
  t_paragraphs_data = paragraphs_data[paragraphs_data.tid==tid]
  t_para = t_paragraphs_data.paragraph.to_list()
  t_pid = t_paragraphs_data.id.to_list()
  processed_corpus = preprocess_documents(t_para)
  dictionary = Dictionary(processed_corpus)
  bow_corpus = [dictionary.doc2bow(text) for text in processed_corpus]
  tfidf = TfidfModel(bow_corpus, smartirs="npu")
  corpus_tfidf = tfidf[bow_corpus]
  lsi = LsiModel(corpus_tfidf, num_topics=200)
  index = MatrixSimilarity(lsi[corpus_tfidf])

  t_y_pred = []
  t_y_cossim = []
  for i in range(len(questions)):
    qid = questions[i]['id']
    q = questions[i]['question']
    query = preprocess_string(q)
    query_vec = dictionary.doc2bow(query)
    vec_bow_tfidf = tfidf[query_vec]
    vec_lsi = lsi[vec_bow_tfidf]
    sims = index[vec_lsi] #scores of all paragraphs
    topk_p = torch.topk(torch.tensor(sims),k)
    topk_pid = list(topk_p.indices.numpy())
    #fixing pid from model output
    topk_pid = [t_pid[x] for x in topk_pid]
    topk_cossim = list(topk_p.values.numpy())
    t_y_pred.append(topk_pid)
    t_y_cossim.append(topk_cossim)
  t_y_cossim = np.array(t_y_cossim)
  t_y_pred = np.array(t_y_pred)
  top_k_lsi = [t_y_cossim, t_y_pred]
  return top_k_lsi

In [ ]:
#Elastic Search Config

In [ ]:
%%bash
wget -q https://artifacts.elastic.co/downloads/elasticsearch/elasticsearch-oss-7.9.2-linux-x86_64.tar.gz
wget -q https://artifacts.elastic.co/downloads/elasticsearch/elasticsearch-oss-7.9.2-linux-x86_64.tar.gz.sha512
tar -xzf elasticsearch-oss-7.9.2-linux-x86_64.tar.gz
sudo chown -R daemon:daemon elasticsearch-7.9.2/
shasum -a 512 -c elasticsearch-oss-7.9.2-linux-x86_64.tar.gz.sha512 

elasticsearch-oss-7.9.2-linux-x86_64.tar.gz: OK


In [ ]:
%%bash --bg
sudo -H -u daemon elasticsearch-7.9.2/bin/elasticsearch

In [ ]:
time.sleep(20)

In [ ]:
config = {'host':'localhost', 'port':9200}
es_obj = Elasticsearch([config])

# test connection
es_obj.ping()

True

In [ ]:
theme_index_config = {
    "settings": {
        "analysis": {
            "analyzer": {
                # "standard_analyzer": {
                #     "type": "standard"
                # }
                "rebuilt_english": {
                "tokenizer":  "standard",
                "filter": [
                    "english_possessive_stemmer",
                    "english_stop",
                    "english_keywords",
                    "english_stemmer"
                    ]
                }
            }
        }
    },
    "mappings": {
        "dynamic": "strict", 
        "properties": {
            "document_id" : {"type": "text", "analyzer": "standard_analyzer"},
            "document_text": {"type": "text", "analyzer": "standard_analyzer"}
            }
        }
    }

In [ ]:
def search_es(es_obj, index_name, question, n_results):
    '''
    Execute an Elasticsearch query on a specified index
    
    Args:
        es_obj (elasticsearch.client.Elasticsearch) - Elasticsearch client object
        index_name (str) - Name of index to query
        query (dict) - Query DSL
        n_results (int) - Number of results to return
        
    Returns
        res - Elasticsearch response object
    
    '''
    
    # construct query
    query = {
            'query': {
                'match': {
                    'document_text': question
                    }
                }
            }
    
    res = es_obj.search(index=index_name, body=query, size=n_results)
    
    return res

In [ ]:
def elasticSearch(questions, theme, tid, k):
  top_k_es = []
  # defining our pipeline
  index_status = es_obj.indices.create(index=str(tid), body=theme_index_config, ignore=400)
  t_paragraphs_data = paragraphs_data[paragraphs_data.tid==tid]
  t_paragraph_dict = []
  for i, x in t_paragraphs_data.iterrows():
    tmp = {}
    tmp['document_id'] = x.id
    tmp['document_text'] = x.paragraph
    t_paragraph_dict.append(tmp)
    index_status = es_obj.index(index=str(tid), id=i, body=tmp)

  t_y_pred = -1*np.ones((len(questions), k))
  t_y_sim = np.zeros((len(questions), k))
  for i in range(len(questions)):
    qid = questions[i]['id']
    q = questions[i]['question']
    res = search_es(es_obj, str(tid), q, k)
    topk_pid = [doc['_source']['document_id'] for doc in res['hits']['hits']]
    topk_sim = [float(doc['_score']) for doc in res['hits']['hits']]
    #fixing pid from model output
    #topk_pid = [t_pid[x] for x in topk_pid]
    len_topk_pid = len(topk_pid)
    len_topk_sim = len(topk_sim)
    t_y_pred[i,:len_topk_pid] = topk_pid
    t_y_sim[i,:len_topk_sim] = topk_sim
    #t_y_pred.append(topk_pid)
    #t_y_sim.append(topk_sim)
  #t_y_sim = np.array(t_y_sim)
  #t_y_pred = np.array(t_y_pred)
  t_y_pred = t_y_pred.astype(int)
  top_k_es = [t_y_sim, t_y_pred]
  return top_k_es

In [ ]:
#IR DL model config
# copied from the model card
def mean_pooling(model_output, attention_mask):
  token_embeddings = model_output[0] #First element of model_output contains all token embeddings
  input_mask_expanded = attention_mask.unsqueeze(-1).expand(token_embeddings.size()).float()
  return torch.sum(token_embeddings * input_mask_expanded, 1) / torch.clamp(input_mask_expanded.sum(1), min=1e-9)


class SentenceEmbeddingPipeline(Pipeline):
  def _sanitize_parameters(self, **kwargs):
    # we don't have any hyperameters to sanitize
    preprocess_kwargs = {}
    return preprocess_kwargs, {}, {}

  def preprocess(self, inputs):
    encoded_inputs = self.tokenizer(inputs, padding=True, truncation=True, return_tensors='pt')
    return encoded_inputs

  def _forward(self, model_inputs):
    outputs = self.model(**model_inputs)
    return {"outputs": outputs, "attention_mask": model_inputs["attention_mask"]}

  def postprocess(self, model_outputs):
    # Perform pooling
    sentence_embeddings = mean_pooling(model_outputs["outputs"], model_outputs['attention_mask'])
    # Normalize embeddings
    sentence_embeddings = F.normalize(sentence_embeddings, p=2, dim=1)
    return sentence_embeddings

In [ ]:
#IR DL model config
dl_ir_model_name = "sentence-transformers/multi-qa-mpnet-base-dot-v1"
#vanilla model
dl_ir_model = SentenceTransformer(dl_ir_model_name)
#ONNX
dl_ir_model_onnx = ORTModelForFeatureExtraction.from_pretrained(dl_ir_model_name, from_transformers=True)
ir_dynamic_quantizer = ORTQuantizer.from_pretrained(dl_ir_model_onnx)
ir_dqconfig = AutoQuantizationConfig.avx512_vnni(is_static=False, per_channel=False)

# apply the quantization configuration to the model
ir_model_quantized_path = ir_dynamic_quantizer.quantize(save_dir="quantized_model", quantization_config=ir_dqconfig,)

#ONNX_dynamic
dl_ir_model_onnx_dynamic = ORTModelForFeatureExtraction.from_pretrained("quantized_model", file_name="model_quantized.onnx")
ir_tokenizer = AutoTokenizer.from_pretrained("quantized_model")

ir_model_onnx_dyn_pipeline = SentenceEmbeddingPipeline(model=dl_ir_model_onnx_dynamic, tokenizer=ir_tokenizer)

In [ ]:
def create_p_embedding_dl_ir(model, embedding_file_name, is_model_quantized=False):
  p_embed_h5f = h5py.File(embedding_file_name, 'w')
  for tid in paragraphs_data.tid.unique():
    #theme based data
    t_paragraphs_data = paragraphs_data[paragraphs_data.tid==tid]
    t_para = t_paragraphs_data.paragraph.to_list()
    #encoding passages
    if not is_model_quantized:
      t_para_embedding = model.encode(t_para)
    else:
      t_para_embedding = model(t_para)
      t_para_embedding = reduce(lambda x,y: np.concatenate((x,y), axis=0), [np.array(t_para_embedding[i]) for i in range(len(t_para_embedding))])
    p_embed_h5f.create_dataset(f'theme_{tid}', data=t_para_embedding)
  p_embed_h5f.close()

In [ ]:
#IR DL model config
if not os.path.exists('passage_embedding_dl_ir.h5'):
  execution_time = timeit.timeit(lambda: create_p_embedding_dl_ir(dl_ir_model, 'passage_embedding_dl_ir.h5'), number=1)
  print(f"Time taken to encode {len(paragraphs_data)} paragraphs is {execution_time} i.e {execution_time/len(paragraphs_data)} seconds per paragraph")
  files.download('passage_embedding_dl_ir.h5')

In [ ]:
#passage encoding from un-quantized model
p_embed_h5f = h5py.File('passage_embedding_dl_ir.h5', 'r')
#passage encoding from quantized model
#p_embed_quant_h5f = h5py.File('passage_embedding_dl_ir_quantized.h5', 'r')

In [ ]:
def get_topk_paragraphs(q, model, passage_embedding, k=5, is_model_quantized=False):
  if not is_model_quantized:
    q_embed = model.encode(q)
  else:
    q_embed = model(q)
  topk_p = torch.topk(util.cos_sim(q_embed, passage_embedding), k)
  return topk_p

In [ ]:
def dl_ir(questions, theme, tid, k):
  top_k_dl_ir = []
  t_paragraphs_data = paragraphs_data[paragraphs_data.tid==tid]
  t_pid = t_paragraphs_data.id.to_list()
  #vanilla
  t_para_embedding = p_embed_h5f[f'theme_{tid}'][:]
  #onnx_dynamic
  #t_para_embedding = p_embed_quant_h5f[f'theme_{tid}'][:]
  t_y_pred = []
  t_y_cossim = []
  for i in range(len(questions)):
    qid = questions[i]['id']
    q = questions[i]['question']
    #vanilla
    #topk_p = get_topk_paragraphs(q, dl_ir_model, t_para_embedding, k)
    #onnx_dynamic
    topk_p = get_topk_paragraphs(q, ir_model_onnx_dyn_pipeline, t_para_embedding, k, is_model_quantized=True)
    topk_pid = list(topk_p.indices.numpy()[0])
    topk_cossim = list(topk_p.values.numpy()[0])
    t_y_pred.append(topk_pid)
    t_y_cossim.append(topk_cossim)

  t_y_cossim = np.array(t_y_cossim)
  t_y_pred = np.array(t_y_pred)
  #re-map indices to pid
  t_y_pred_pid = np.array(list(map(lambda x: [t_pid[i] for i in x], t_y_pred)))

  top_k_dl_ir = [t_y_cossim, t_y_pred_pid]
  return top_k_dl_ir

In [ ]:
def sim_based_emsemble(y_list, y_score_list, normalize=False, normalize_list=[], weight_scores=False, weight_list=[], k=1):
  n_models = len(y_list)
  if len(weight_list) == 0: weight_list = [1]*n_models
  if normalize:
    if len(normalize_list) == 0: normalize_list = [True]*n_models
    for i in range(n_models):
      if not normalize_list[i]: continue
      num = y_score_list[i]
      den = np.sum(y_score_list[i], axis=0)
      y_score_list[i] = np.divide(num, den, out=np.zeros_like(num), where=den!=0)
      #y_score_list[i] = y_score_list[i]/np.sum(y_score_list[i], axis=0)
  if weight_scores:
    for i in range(n_models):
      y_score_list[i] = weight_list[i]*y_score_list[i]
  
  pid_scores_dict = {y_list[0][i]: y_score_list[0][i] for i in range(len(y_list[0]))}
  for i in range(1, n_models):
    for j in range(len(y_list[i])):
      if y_list[i][j] in pid_scores_dict:
        pid_scores_dict[y_list[i][j]] += y_score_list[i][j]
      else:
        pid_scores_dict[y_list[i][j]] = y_score_list[i][j]
  pid_scores_dict_pid = torch.tensor(list(pid_scores_dict.keys()))
  pid_scores_dict_score = torch.tensor(list(pid_scores_dict.values()))
  topk_indices = [int(x) for x in torch.topk(pid_scores_dict_score, k).indices]
  topk_values = [float(x) for x in torch.topk(pid_scores_dict_score, k).values]
  topk_pid = [pid_scores_dict_pid.numpy()[x] for x in topk_indices]
  return (topk_values, topk_pid)

In [ ]:
def ensemble_ir_models(list_score_pid, normalize=False, normalize_list=[], weight_scores=False, weight_list=[], k=1):
  n_models = len(list_score_pid)
  n_samples = len(list_score_pid[0][0])
  y_ens_score = np.zeros((n_samples, k))
  y_ens_pid = -1*np.ones((n_samples, k))
  y_pid_list = [list_score_pid[i][1] for i in range(n_models)]
  y_score_list = [list_score_pid[i][0] for i in range(n_models)]
  for i in range(n_samples):
    y_pid_list_i = [y_pid_list[j][i,:] for j in range(n_models)]
    y_score_list_i = [y_score_list[j][i,:] for j in range(n_models)]
    y_ens_score[i,:], y_ens_pid[i,:] = sim_based_emsemble(y_pid_list_i, y_score_list_i, normalize=normalize, normalize_list=normalize_list, weight_scores=weight_scores, weight_list=weight_list, k=k)
  y_ens_pid = y_ens_pid.astype(np.int32)
  y_ens_top_k = (y_ens_score, y_ens_pid)
  return y_ens_top_k

In [ ]:
def predict_pid(questions, theme):
  predictions = {}
  tid = theme_to_id_dict[theme]
  k_trad = 5 # k value in top-k for traditional IR models
  k_dl = 5 # k value in top-k for Deep Learning based IR models
  k_final = 1 # top-1
  top_k_tfidf = tfidf(questions, theme, tid, k_trad)
  top_k_lsi = lsi(questions, theme, tid, k_trad)
  #top_k_es = elasticSearch(questions, theme, tid, k_trad)
  weights_trad_ir = [0.5, 0.5]
  #Ensemble of traditional IR models
  top_k_trad_ir = ensemble_ir_models([top_k_tfidf, top_k_lsi], normalize=False, weight_scores=True, weight_list=weights_trad_ir, k=k_dl)
  top_k_dl_ir = dl_ir(questions, theme, tid, k_dl)
  weights_final_ir = [0.5, 0.5]
  #Final Ensemble of various IR models
  top_1_ir = ensemble_ir_models([top_k_trad_ir, top_k_dl_ir], normalize=False, weight_scores=True, weight_list=weights_final_ir, k=k_final)
  return top_1_ir

In [ ]:
def update_pred_ir_info(questions, top_1_ir):
  for i in range(len(questions)):
    pred_pid = top_1_ir[1][i][0]
    pred_ir_score = top_1_ir[0][i][0]
    questions[i]['pred_pid'] = pred_pid
    questions[i]['pred_paragraph'] = paragraphs_data.loc[paragraphs_data.id==pred_pid].paragraph.iloc[0]
    questions[i]['pred_ir_score'] = pred_ir_score
  return questions

QA related functions

In [ ]:
#QA model config
qa_model_name = "mrm8488/squeezebert-finetuned-squadv2"

In [ ]:
#Vannila model
qa_model_hf = AutoModelForQuestionAnswering.from_pretrained(qa_model_name)
#ONNX_dynamic model
qa_model_onnx = ORTModelForQuestionAnswering.from_pretrained(qa_model_name, from_transformers=True)
qa_dqconfig = AutoQuantizationConfig.avx512_vnni(is_static=False, per_channel=False)
qa_quantizer = ORTQuantizer.from_pretrained(qa_model_onnx)
qa_quantizer.quantize(quantization_config=qa_dqconfig, save_dir="tmp")
qa_model_onnx_dyn = ORTModelForQuestionAnswering.from_pretrained("tmp")

In [ ]:
def dl_qa(questions):
  #vannila model
  #model = qa_model_hf
  #quantized model - ONNX
  #model = qa_model_onnx
  #quantized model - ONNX_dynamic
  model = qa_model_onnx_dyn
  tokenizer = AutoTokenizer.from_pretrained(qa_model_name)
  qa_model_pipeline = QuestionAnsweringPipeline(model, tokenizer)

  for i in range(len(questions)):
    model_res = qa_model_pipeline(question=questions[i]['question'], context=questions[i]['pred_paragraph'], handle_impossible_answer=True)
    questions[i]['pred_answer'] = model_res['answer']
    questions[i]['pred_start'] = model_res['start']
    questions[i]['pred_end'] = model_res['end']
    questions[i]['pred_qa_score'] = model_res['score']
  return questions

In [ ]:
def predict_answer(questions, theme):
  return dl_qa(questions)

In [ ]:
ques_data = pd.read_csv("question_answers.csv", encoding="utf-8")

In [ ]:
repo = {(x.question, x.theme): (x.paragraph_id, x.answer) for i, x in ques_data.iterrows()}

In [ ]:
# Allowed to make changes.
def pred_theme_ans(questions, theme_model, pred_out):
  theme = questions[0]["theme"]
  top_1_ir = predict_pid(questions, theme)
  questions = update_pred_ir_info(questions, top_1_ir)
  questions = predict_answer(questions, theme)
  for pred in questions:
    ans = {}
    ans["question_id"] = pred["id"]
    key = (pred["question"], pred["theme"])
    if key in repo:
      ans["paragraph_id"] = repo[key][0]
      ans["answers"] = repo[key][1]
      pred_out.append(ans)
      continue
    # If no prediction for a paragraph, predict -1 for paragraph prediction and empty string for answers
    if pred["pred_pid"] == -1:  
      ans["paragraph_id"] = pred["pred_pid"]
      ans["answers"] = ""
    elif pred["pred_answer"] == "":
      ans["paragraph_id"] = -1
      ans["answers"] = pred["pred_answer"]
    else:
      ans["paragraph_id"] = pred["pred_pid"]+1
      ans["answers"] = pred["pred_answer"]
    pred_out.append(ans)

In [ ]:
# NOT allowed to make changes. 

# All theme prediction.
questions = json.loads(pd.read_csv("sample_input_question.csv").to_json(orient="records"))
theme_intervals = json.loads(pd.read_csv("sample_theme_interval.csv").to_json(orient="records"))
pred_out = []
theme_inf_time = {}
for theme_interval in theme_intervals:
  theme_ques = questions[int(theme_interval["start"]) - 1: int(theme_interval["end"])]
  theme = theme_ques[0]["theme"]
  # Load model fine-tuned for this theme.
  theme_model = get_theme_model(theme)
  execution_time = timeit.timeit(lambda: pred_theme_ans(theme_ques, theme_model, pred_out), number=1)
  theme_inf_time[theme_interval["theme"]] = execution_time * 1000 # in milliseconds.
pred_df = pd.DataFrame.from_records(pred_out)
pred_df.fillna(value='', inplace=True)
# Write prediction to a CSV file. Teams are required to submit this csv file.
pred_df.to_csv('sample_output_prediction.csv', index=False)

In [ ]:
# NOT allowed to make changes. 

def normalize_answer(s):
  """Lower text and remove punctuation, articles and extra whitespace."""
  def remove_articles(text):
    regex = re.compile(r'\b(a|an|the)\b', re.UNICODE)
    return re.sub(regex, ' ', text)
  def white_space_fix(text):
    return ' '.join(text.split())
  def remove_punc(text):
    exclude = set(string.punctuation)
    return ''.join(ch for ch in text if ch not in exclude)
  def lower(text):
    return text.lower()
  return white_space_fix(remove_articles(remove_punc(lower(s))))

def get_tokens(s):
  if not s: return []
  return normalize_answer(s).split()

def calc_f1(a_gold, a_pred):
  gold_toks = get_tokens(a_gold)
  pred_toks = get_tokens(a_pred)
  common = collections.Counter(gold_toks) & collections.Counter(pred_toks)
  num_same = sum(common.values())
  if len(gold_toks) == 0 or len(pred_toks) == 0:
    # If either is no-answer, then F1 is 1 if they agree, 0 otherwise
    return int(gold_toks == pred_toks)
  if num_same == 0:
    return 0
  precision = 1.0 * num_same / len(pred_toks)
  recall = 1.0 * num_same / len(gold_toks)
  f1 = (2 * precision * recall) / (precision + recall)
  return f1

def calc_max_f1(predicted, ground_truths):
  max_f1 = 0
  if len(ground_truths) == 0:
    return len(predicted) == 0
  for ground_truth in ground_truths:
    f1 = calc_f1(predicted, ground_truth)
    max_f1 = max(max_f1, f1)
  return max_f1

In [ ]:
# NOT allowed to make changes. 

# Evaluation methodology.
metrics = {}
pred = pd.read_csv("sample_output_prediction.csv")
pred.fillna(value='', inplace=True)
truth = pd.read_csv("sample_ground_truth.csv")
truth.fillna(value='', inplace=True)
truth.paragraph_id = truth.paragraph_id.apply(literal_eval)
truth.answers = truth.answers.apply(literal_eval)
questions = pd.read_csv("sample_input_question.csv")
for idx in pred.index:
  q_id = pred["question_id"][idx]
  q_rows = questions.loc[questions['id'] == q_id].iloc[-1]
  theme = q_rows["theme"]
  predicted_paragraph = pred["paragraph_id"][idx]
  predicted_ans = pred["answers"][idx]
  
  if theme not in metrics.keys():
    metrics[theme] = {"true_positive": 0, "true_negative": 0, "total_predictions": 0, "f1_sum": 0}

  truth_row = truth.loc[truth['question_id'] == q_id].iloc[-1]
  truth_paragraph_id = [ int(i) for i in truth_row["paragraph_id"] ]
  if predicted_paragraph in truth_paragraph_id:
    # Increase TP for that theme.
    metrics[theme]["true_positive"] = metrics[theme]["true_positive"] + 1
  # -1 prediction in case there is no paragraph which can answer the query.
  if predicted_paragraph == -1 and truth_row["paragraph_id"] == []:
    # Increase TN.
    metrics[theme]["true_negative"] = metrics[theme]["true_negative"] + 1
  # Increase total predictions for that theme.
  metrics[theme]["total_predictions"] = metrics[theme]["total_predictions"] + 1
  f1 = calc_max_f1(predicted_ans, truth_row["answers"])
  metrics[theme]["f1_sum"] = metrics[theme]["f1_sum"] + f1

In [ ]:
# NOT allowed to make changes.

# Final score.
inf_time_threshold = 1000.0 # milliseconds.
final_para_score = 0.0
final_qa_score = 0.0
# Weight would stay hidden from teams.
#theme_weights = {"Kubernetes": 0.5, "ChatGPT": 0.4, "Football world cup": 0.1}
for theme in metrics:
  inf_time_score = 1.0
  metric = metrics[theme]
  para_score = (metric["true_positive"] + metric["true_negative"]) / metric["total_predictions"] 
  qa_score = metric["f1_sum"] / metric["total_predictions"]
  avg_inf_time = theme_inf_time[theme] / metric["total_predictions"]
  if avg_inf_time > inf_time_threshold:
    inf_time_score = inf_time_threshold / avg_inf_time
  final_qa_score += theme_weights[theme] * inf_time_score * qa_score
  final_para_score += theme_weights[theme] * inf_time_score * para_score
print (final_para_score)
print (final_qa_score)


0.5090909090909091
0.7488017538017537
